## Govt Contracts 

In [1]:
from dotenv import load_dotenv
import os
import requests

from typing import List, Optional, Union
from pydantic import BaseModel, Field, HttpUrl, TypeAdapter

In [2]:
load_dotenv()


True

- [SBA Info](https://www.sba.gov/federal-contracting/contracting-guide/how-win-contracts)
- [Hubzone Map](https://maps.certify.sba.gov/hubzone/map#center=44.722800,-103.249700&zoom=4)
- [DBSB](https://dsbs.sba.gov/search/)
- [GSA API Gateway](https://open.gsa.gov/api/)
- [Opportunities Public API](https://open.gsa.gov/api/get-opportunities-public-api/)

Interfacing System Name: COM

System Description and Function
```
I would like to interface with SAM.gov's Opportunity Management API for the primary purpose of extracting and filtering public contract opportunities. I will utilize advanced algorithms to sort opportunities based on predefined criteria such as industry categories, contract value, and geographical location. The aim is to streamline our procurement process by quickly identifying and presenting the most relevant opportunities to our team. The system will include robust security measures to ensure data integrity and compliance with federal data access regulations. It will be regularly updated to maintain compatibility with SAM.gov API developments and security requirements.

In [3]:
SAM_PUBLIC_API_KEY = os.environ.get("SAM_PUBLIC_API_KEY")
base_url = "https://api.sam.gov/opportunities/v2/search"

In [4]:
res = requests.get(f"https://api.sam.gov/opportunities/v2/search?api_key={SAM_PUBLIC_API_KEY}&postedFrom=01/01/2024&postedTo=12/31/2024&ptype=o&limit=1000")

In [7]:
res.json()["opportunitiesData"]

[{'noticeId': 'fff48e45353548f78bc780ccc952e4ad',
  'title': 'ANT ASSY KUBND OMNI',
  'solicitationNumber': 'N0010424QUD44',
  'fullParentPathName': 'DEPT OF DEFENSE.DEPT OF THE NAVY.NAVSUP.NAVSUP WEAPON SYSTEMS SUPPORT.NAVSUP WSS MECHANICSBURG.NAVSUP WEAPON SYSTEMS SUPPORT MECH',
  'fullParentPathCode': '017.1700.NAVSUP.NAVSUP WSS.NAVSUP WSS MECH.N00104',
  'postedDate': '2024-02-29',
  'type': 'Solicitation',
  'baseType': 'Solicitation',
  'archiveType': 'auto15',
  'archiveDate': '2024-03-26',
  'typeOfSetAsideDescription': None,
  'typeOfSetAside': '',
  'responseDeadLine': '2024-03-11T16:30:00-04:00',
  'naicsCode': '334220',
  'naicsCodes': ['334220'],
  'classificationCode': '5985',
  'active': 'Yes',
  'award': None,
  'pointOfContact': [{'fax': None,
    'type': 'primary',
    'email': 'BENJAMIN.SPEROS@NAVY.MIL',
    'phone': None,
    'title': None,
    'fullName': 'Telephone: 7176055150'}],
  'description': 'https://api.sam.gov/prod/opportunities/v1/noticedesc?noticeid=fff4

### Breaking Down Codes:

- [Blog Post about codes](https://www.wolterskluwer.com/en/expert-insights/how-to-decipher-government-codes-relating-to-contracting)
- [Census NAICS Category Lookup](https://www.census.gov/naics/?input=web&year=2022)

## Schema

In [46]:
class Award(BaseModel):
    awardee: Optional[dict] = None

class PointOfContact(BaseModel):
    fax: Optional[str] = None
    type: str
    email: str
    phone: Optional[str] = None
    title: Optional[str] = None
    fullName: str

class Address(BaseModel):
    zipcode: Optional[str]
    city: str
    countryCode: str
    state: str

class PlaceOfPerformance(BaseModel):
    city: Optional[Union[dict, str]] = None
    state: Optional[Union[dict, str]] = None
    country: Optional[Union[dict, str]] = None
    zip: Optional[str] = None

class Link(BaseModel):
    rel: str
    href: HttpUrl

class Notice(BaseModel):
    noticeId: str = Field(..., alias='noticeId')
    title: str
    solicitationNumber: str
    fullParentPathName: str
    fullParentPathCode: str
    postedDate: str
    type: str
    baseType: str
    archiveType: str
    archiveDate: str
    typeOfSetAsideDescription: Optional[str] = None
    typeOfSetAside: Optional[str] = None
    responseDeadLine: str
    naicsCode: str
    naicsCodes: List[str]
    classificationCode: str
    active: str
    award: Optional[Award] = None
    pointOfContact: List[PointOfContact]
    description: HttpUrl
    organizationType: str
    officeAddress: Optional[Address] = None
    placeOfPerformance: Optional[PlaceOfPerformance] = None
    additionalInfoLink: Optional[HttpUrl] = None
    uiLink: HttpUrl
    links: List[Link]
    resourceLinks: Optional[List[HttpUrl]] = None

    class Config:
        populate_by_name = True

In [47]:
json_dict = res.json()["opportunitiesData"]

In [48]:
notice = TypeAdapter(Notice).validate_python(json_dict[0])

In [49]:
Notice.model_validate(json_dict[0])

Notice(noticeId='d524f3b99db14bc4b0ce4f3bb61ea6c7', title='Elevators Maintenance Services', solicitationNumber='RFQ19J01024Q0010', fullParentPathName='STATE, DEPARTMENT OF.STATE, DEPARTMENT OF.US EMBASSY AMMAN', fullParentPathCode='019.1900.19J010', postedDate='2024-02-28', type='Solicitation', baseType='Solicitation', archiveType='auto15', archiveDate='2024-03-22', typeOfSetAsideDescription=None, typeOfSetAside=None, responseDeadLine='2024-03-07T16:00:00+03:00', naicsCode='811', naicsCodes=['811'], classificationCode='Z1NZ', active='Yes', award=None, pointOfContact=[PointOfContact(fax='', type='primary', email='copelandJM@state.gov', phone='', title=None, fullName='Jessi Copeland'), PointOfContact(fax='', type='secondary', email='zabanehrg@state.gov', phone='96265906060', title=None, fullName='Rula Zabaneh')], description=Url('https://api.sam.gov/prod/opportunities/v1/noticedesc?noticeid=d524f3b99db14bc4b0ce4f3bb61ea6c7'), organizationType='OFFICE', officeAddress=Address(zipcode='2052

In [50]:
def validate_notices(notices: List[dict]):
    return [Notice.model_validate(item) for item in notices]

In [51]:
validate_notices(json_dict)

ValidationError: 2 validation errors for Notice
officeAddress.zipcode
  Field required [type=missing, input_value={'city': 'PRETORIA', 'countryCode': 'ZAF'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing
officeAddress.state
  Field required [type=missing, input_value={'city': 'PRETORIA', 'countryCode': 'ZAF'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing